## 1. Setup and Configuration

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 10

print("✅ Libraries loaded successfully!")
print(f"ctx_date_month: {datetime.now().strftime('%Y-%m-%d')}")

✅ Libraries loaded successfully!
ctx_date_month: 2026-01-27


## 2. Load and Prepare Data

Load your forecast data with actual demand for validation.

In [17]:
# Import required libraries
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..",))
sys.path.append(project_root)
from src.core.models.xgboost import XgboostModel
import pandas as pd

from src.core.config_loader import ConfigLoader
config_loader = ConfigLoader()


model = XgboostModel().load_model(
    '/Users/anapreciado/Desktop/porygon-demand-forecasting/src/artifacts/models/xgboost/20260127_154750/model_0.pickle'
)

INFO:src.core.models.xgboost:Model loaded from /Users/anapreciado/Desktop/porygon-demand-forecasting/src/artifacts/models/xgboost/20260127_154750/model_0.pickle


In [18]:
df_new = model.validation_df
df_new["predicted"] = model.predict(df_new[model.features])
df = df_new.copy()
df["benchmark"] = df[config_loader.benchmark_col]

# Filter nulls
df = df[~df["benchmark"].isna()]

df_complete_skus = df.groupby(["ctx_item_id"]).count()[["ctx_date_month"]].reset_index()
df = df.merge(
    df_complete_skus[df_complete_skus["ctx_date_month"]==6][["ctx_item_id"]],
    on=["ctx_item_id"],
    how="inner"
)
df = df.sort_values(['ctx_store_id', 'ctx_item_id', 'ctx_date_month']).reset_index(drop=True)

df = df[
    ~df['fea_item_monthly_sales_roll_std_3_months'].isna()
]

## 3. Define Cost Parameters

Set up inventory holding costs and stockout penalty costs.

In [19]:
# Cost parameters
COST_CONFIG = {
    'unit_cost_percent_of_price': 0.10,
    'holding_cost_per_unit_per_month': 0.20,  # 2% of unit cost per month
    'lead_time_months': 3,                    # Orders arrive in 3 months
}

print("💰 COST CONFIGURATION")
print("=" * 80)
print(f"Holding Cost Rate:        {COST_CONFIG['holding_cost_per_unit_per_month']*100:.1f}% of unit cost per month")
print(f"Lead Time:                {COST_CONFIG['lead_time_months']} months")
print("=" * 80)

# Add calculated cost columns to dataframe
df['unit_cost'] = df['fea_item_store_price_avg'] * COST_CONFIG['unit_cost_percent_of_price']
df['holding_cost_per_unit_per_month'] = df['unit_cost'] * COST_CONFIG['holding_cost_per_unit_per_month']
df['gross_margin_per_unit'] = df['fea_item_store_price_avg'] - df['unit_cost']
df['benchmark_with_ss'] = df['benchmark'] + (df['fea_item_monthly_sales_roll_std_3_months'] * 1.55 * np.sqrt(COST_CONFIG['lead_time_months'])) # df['benchmark'] + (df['fea_item_monthly_sales_roll_std_3_months'] * 1.55 * np.sqrt(COST_CONFIG['lead_time_months']))
df['benchmark_with_ss'] = df['benchmark_with_ss'].round().astype(int)
df['predicted'] = df['predicted'].round().astype(int)
print("\n✅ Cost parameters configured!")

💰 COST CONFIGURATION
Holding Cost Rate:        20.0% of unit cost per month
Lead Time:                3 months

✅ Cost parameters configured!


## 4. Inventory Simulation Function

Core simulation logic that tracks inventory, orders, stockouts, and costs over time.

In [20]:
# todo: need to define initial inventory parameters.

def simulate_inventory_costs(df, forecast_col, lead_time_months=3, simulation_start_date=None):
    """
    Simulate inventory management using forecast to determine order quantities.
    
    Parameters:
    -----------
    df : DataFrame
        Data with columns: date, store_id, sku, actual_demand, forecast_col, 
                          price, unit_cost, holding_cost_per_unit_per_month
    forecast_col : str
        Column name containing the 3-month forecast (demand + safety stock)
    lead_time_months : int
        Number of months for orders to arrive (default 3)
    simulation_start_date : str or None
        Date to start tracking costs (before this, we just build up pending orders)
        
    Returns:
    --------
    DataFrame with simulation results including costs
    """
    
    results = []
    # Group by store-SKU.
    for (store, sku), group in df.groupby(['ctx_store_id', 'ctx_item_id']):
        # Set inventory transit tracking.
        group["ctx_date_month"] = pd.to_datetime(group["ctx_date_month"])
        group = group.sort_values('ctx_date_month').reset_index(drop=True)
        placed_orders = {}
        current_month = group["ctx_date_month"].iloc[0]
        initial_inventory = group['benchmark_with_ss'].iloc[0]
        holding_cost_rate = group['holding_cost_per_unit_per_month'].iloc[0]
        price = group['fea_item_store_price_avg'].iloc[0]
        unit_cost = group['unit_cost'].iloc[0]
        for idx, row in group.iterrows():
            current_month = row['ctx_date_month']
            actual_monthly_demand = row['fea_item_monthly_sales']
            projected_demand_in_3_months = row[forecast_col]

            # update inventory in transit
            arriving_inventory = placed_orders.get(current_month - pd.DateOffset(months=2),0)
            transit_inventory = placed_orders.get(current_month - pd.DateOffset(months=1),0)

            # inventory arriving within 3 months.
            inventory_arriving_within_3_months = (
                placed_orders.get(current_month, 0) 
                + transit_inventory
                + arriving_inventory
            )

            # How much to order.
            ordered_qty = max(
                0, projected_demand_in_3_months - (initial_inventory + inventory_arriving_within_3_months)
            )

            # Update inventory in transit.
            placed_orders.update(
                {current_month: ordered_qty}
            )

            # Ending Iventory
            ending_inventory = max(0, initial_inventory + arriving_inventory - actual_monthly_demand)

            # Average inventory
            average_inventory = (initial_inventory + ending_inventory) / 2

            # Calculate units sold and stockouts.
            units_sold = min(initial_inventory + arriving_inventory, actual_monthly_demand)
            stockout_units = max(0, actual_monthly_demand - (initial_inventory + arriving_inventory))

            # Calculate inventory costs (fees and holding).
            holding_costs = ending_inventory * holding_cost_rate
            held_inventory_unit_cost = ending_inventory * unit_cost
            stockout_profit_loss = (stockout_units * price) - (stockout_units * unit_cost)
            total_costs = holding_costs + held_inventory_unit_cost

            # Calculate revenue and profit.
            revenue = units_sold * price
            cogs = units_sold * unit_cost
            gross_profit = revenue - cogs
            net_profit = gross_profit - total_costs

            # Record results
            results.append({
                'date': current_month,
                'store_id': store,
                'sku': sku,

                # Inventory tracking
                'initial_inventory': initial_inventory,
                'arriving_inventory': arriving_inventory,
                'transit_inventory': transit_inventory,
                'ending_inventory': ending_inventory,
                'average_inventory': average_inventory,
                
                # Demand and fulfillment
                'actual_monthly_demand': actual_monthly_demand,
                'projected_demand_3m': projected_demand_in_3_months,
                'units_sold': units_sold,
                'stockout_units': stockout_units,

                'units_sold': units_sold,
                'stockout_units': stockout_units,
                'fill_rate': (units_sold / actual_monthly_demand * 100) if actual_monthly_demand > 0 else 1,
                
                # Orders
                'ordered_qty': ordered_qty,
                
                # Costs
                'holding_cost_per_unit': holding_cost_rate,
                'pricce': price,
                'unit_cost': unit_cost,
                'holding_cost': holding_costs,
                'held_inventory_unit_cost': held_inventory_unit_cost,
                'total_cost': total_costs,
                
                # Revenue
                'revenue': revenue,
                'cogs': cogs,
                'gross_profit': gross_profit,
                'net_profit': net_profit,
                'stockout_profit_loss': stockout_profit_loss,
            })

            initial_inventory = ending_inventory
    
    return pd.DataFrame(results)


print("✅ Simulation function defined!")

✅ Simulation function defined!


## 5. Run Simulations

Compare your forecast against benchmark approaches.

In [21]:
print("=" * 80)
print("🔄 RUNNING SIMULATIONS")
print("=" * 80)

# Simulation 1: Your Forecast (3-month demand + safety stock)
forecast_to_simulate = "benchmark_with_ss"
print(f"Running simulation for {forecast_to_simulate}...")
df_sample = df
results_benchmark = simulate_inventory_costs(
    df_sample, 
    forecast_col=forecast_to_simulate,
    lead_time_months=COST_CONFIG['lead_time_months'],
)
results_benchmark['model'] = 'benchmark'
print("   ✓ Complete\n")

# Simulation 2: Benchmark Forecast (rolling average + safety stock)
forecast_to_simulate = "predicted"
print(f"Running simulation for {forecast_to_simulate}...")
results_predicted = simulate_inventory_costs(
    df_sample, 
    forecast_col='predicted',
    lead_time_months=COST_CONFIG['lead_time_months'],
)
results_predicted['model'] = 'prediction'
print("   ✓ Complete\n")

# Combine results
all_results = pd.concat([results_benchmark, results_predicted], ignore_index=True)
# 
print("=" * 80)
print(f"✅ Simulations complete!")
print(f"   Total records: {len(all_results):,}")

🔄 RUNNING SIMULATIONS
Running simulation for benchmark_with_ss...
   ✓ Complete

Running simulation for predicted...
   ✓ Complete

✅ Simulations complete!
   Total records: 1,560


In [22]:
##results_predicted.sort_values(['store_id', 'sku', 'date']).to_csv(
##    '/Users/anapreciado/Desktop/simulation_results_ml.csv'
##)

## 6. Cost Analysis and Comparison

Calculate total costs and validate optimization.

In [23]:
all_results["stockout_period"] = 0
all_results.loc[all_results["stockout_units"] > 0, "stockout_period"] = 1
df_stockout_periods = all_results.groupby(["sku", "date", "model"])[["stockout_period"]].max().reset_index()
df_service_level = df_stockout_periods.groupby(["sku", "model"]).agg(
    total_periods = ('stockout_period', 'count'),
    stockout_periods = ('stockout_period', 'sum')
).reset_index()
df_service_level["service_level"] = (1 - (df_service_level["stockout_periods"] / df_service_level["total_periods"])) * 100
df_service_level.groupby(["model"])[["service_level"]].mean()

,service_level
model,
benchmark,22.98
prediction,75.14


In [24]:
print("=" * 80)
print("💰 COST OPTIMIZATION ANALYSIS")
print("=" * 80)

# Aggregate costs by model
cost_summary = all_results.groupby('model').agg({
    'holding_cost': 'sum',
    'held_inventory_unit_cost': 'sum',
    'total_cost': 'sum',
    'revenue': 'sum',
    'gross_profit': 'sum',
    'net_profit': 'sum',
    'units_sold': 'sum',
    'stockout_units': 'sum',
    'stockout_profit_loss': 'sum',
    'actual_monthly_demand': 'sum',
    'fill_rate': 'mean',
    'average_inventory': 'mean',
}).round(2)

cost_summary['fill_reate'] = (cost_summary['units_sold'] / cost_summary['actual_monthly_demand'] * 100).round(2)
#cost_summary['service_level'] = (cost_summary['units_sold'] / cost_summary['actual_monthly_demand'] * 100).round(2)
# Display results
print("\n📊 COMPREHENSIVE COST COMPARISON\n")

for model in ['prediction', "benchmark"]:
    data = cost_summary.loc[model]
    
    print(f"\n{'='*60}")
    print(f"{model.upper()}")
    print(f"{'='*60}")
    print(f"{'COSTS:':<30}")
    print(f"  Holding Costs:                ${data['holding_cost']:>15,.0f}")
    print(f"  Held Inventory Unit Cost:     ${data['held_inventory_unit_cost']:>15,.0f}")
    print(f"  {'─'*44}")
    print(f"")
    print(f"  TOTAL COSTS:                  ${data['total_cost']:>15,.0f}")
    print()
    print(f"{'PERFORMANCE:':<30}")
    print(f"  Revenue:                      ${data['revenue']:>15,.0f}")
    print(f"  Gross Profit:                 ${data['gross_profit']:>15,.0f}")
    print(f"  Net Profit (after costs):     ${data['net_profit']:>15,.0f}")
    print(f"  Fill Rate:                     {data['fill_reate']:>15.2f}%")
    print(f"  Units Sold:                    {data['units_sold']:>15,.0f}")
    print(f"  Stockout Units:                {data['stockout_units']:>15,.0f}")
    print(f"  Average Inventory:             {data['average_inventory']:>15,.0f}")
    print(f"  Stockout Opportunity Cost:    ${data['stockout_profit_loss']:>15,.0f}")

# Calculate comparative metrics
your_costs = cost_summary.loc['prediction']
benchmark_costs = cost_summary.loc['benchmark']

cost_diff = your_costs['total_cost'] - benchmark_costs['total_cost']
cost_diff_pct = (cost_diff / benchmark_costs['total_cost'] * 100)
profit_diff = your_costs['net_profit'] - benchmark_costs['net_profit']
netprofit_diff_pct = (profit_diff / benchmark_costs['net_profit'] * 100)
service_diff = your_costs['fill_reate'] - benchmark_costs['fill_reate']

print(f"\n{'='*60}")
print("📈 COMPARATIVE ANALYSIS: YOUR FORECAST vs BENCHMARK")
print(f"{'='*60}")
print(f"Total Cost Difference:                   ${cost_diff:>12,.0f} ({cost_diff_pct:+.2f}%)")
print(f"Holding Cost Difference:                 ${your_costs['holding_cost'] - benchmark_costs['holding_cost']:>12,.0f}")
print(f"Stockout Opportunity Cost Difference:    ${your_costs['stockout_profit_loss'] - benchmark_costs['stockout_profit_loss']:>12,.0f}")
print(f"Net Profit Difference:                   ${profit_diff:>12,.0f} ({netprofit_diff_pct:+.2f}%)")
print(f"Fill Rate Difference:                     {service_diff:>12.2f} percentage points")

print(f"\n{'='*60}")
if profit_diff > 0:
    savings = abs(profit_diff)
    print(f"✅ SUCCESS! Your forecast increasese net profit by ${savings:,.0f} ({abs(netprofit_diff_pct):.2f}%)")
    print(f"   Cost optimization is VALIDATED ✓")
elif profit_diff < 0:
    excess = profit_diff
    print(f"⚠️  WARNING: Your forecast DECREASES net profit by ${excess:,.0f} ({netprofit_diff_pct:.2f}%)")
    print(f"   Consider adjusting forecast parameters")
else:
    print(f"➡️  Both models have similar costs")

print(f"{'='*60}\n")

💰 COST OPTIMIZATION ANALYSIS

📊 COMPREHENSIVE COST COMPARISON


PREDICTION
COSTS:                        
  Holding Costs:                $          2,096
  Held Inventory Unit Cost:     $         10,479
  ────────────────────────────────────────────

  TOTAL COSTS:                  $         12,575

PERFORMANCE:                  
  Revenue:                      $         86,280
  Gross Profit:                 $         77,652
  Net Profit (after costs):     $         65,078
  Fill Rate:                               73.58%
  Units Sold:                             23,538
  Stockout Units:                          8,453
  Average Inventory:                          37
  Stockout Opportunity Cost:    $         23,085

BENCHMARK
COSTS:                        
  Holding Costs:                $            102
  Held Inventory Unit Cost:     $            509
  ────────────────────────────────────────────

  TOTAL COSTS:                  $            611

PERFORMANCE:                  
  Rev